In [1]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression, LinearRegression
# from sklearn.metrics import accuracy_score, mean_squared_error
# from colorama import Fore, Style
# from tabulate import tabulate
# from prettytable import PrettyTable

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from colorama import Fore, Style
from prettytable import PrettyTable

df_existing = pd.read_excel('NBA2324.xlsx')

# checking if the 'Win' column exists before proceeding
winning_team = 'Win'  # Update with the actual column name
if winning_team in df_existing.columns:
    # creating a binary target variable indicating whether the home team wins
    df_existing['HomeWin'] = (df_existing['Home team'] == df_existing[winning_team]).astype(int)

    # specifying the categorical columns for one-hot encoding
    categorical_columns = ['Away team', 'Home team', 'Overtime', 'Arena']

    # checking if categorical columns exist in the DataFrame
    missing_columns = [col for col in categorical_columns if col not in df_existing.columns]

    if not missing_columns:
        # extracting the necessary columns for the model
        X_completed = df_existing[categorical_columns]
        y_completed = df_existing['HomeWin']

        # one-hot encode categorical columns
        X_completed_encoded = pd.get_dummies(X_completed, drop_first=True)

        # splitting the completed data into training and testing sets
        X_train_completed, _, y_train_completed, _ = train_test_split(
            X_completed_encoded, y_completed, test_size=0.2, random_state=42
        )

        # choosing a model for completed games (Logistic Regression in this example)
        model_completed = LogisticRegression()

        # training the model on the completed data
        model_completed.fit(X_train_completed, y_train_completed)
    else:
        print(f"Error: Columns {missing_columns} not found in DataFrame.")
else:
    print(f"Error: Column '{winning_team}' not found in DataFrame.")

# upcoming game data
upcoming_game_data = {
    'Away team': ['Indiana Pacers', 'Philadelphia 76ers', 'Chicago Bulls', 'San Antonio Spurs', 'New York Knicks', 'New Orleans Pelicans', 'Denver Nuggets', 'Washington Wizards', 'Golden State Warriors', 'Minnesota Timberwolves'],
    'Home team': ['Charlotte Hornets', 'Cleveland Cavaliers', 'Atlanta Hawks', 'Toronto Raptors', 'Houston Rockets', 'Memphis Grizzlies', 'Milwaukee Bucks', 'Dallas Mavericks', 'Utah Jazz', 'Los Angeles Clippers'],
    'Overtime': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # placeholder for upcoming games
    'Arena': ['Spectrum Center', 'Rocket Mortgage Fieldhouse', 'State Farm Arena', 'Scotiabank Arena', 'Toyota Center', 'FedEx Forum', 'Fiserv Forum', 'American Airlines Center', 'Delta Center', 'Crypto.com Arena'],
}

# creating a DataFrame for the upcoming game
df_upcoming_game = pd.DataFrame(upcoming_game_data)

# ensuring the columns in df_upcoming_game_encoded match the columns used during training
df_upcoming_game_encoded = pd.get_dummies(df_upcoming_game)
missing_columns = set(X_train_completed.columns) - set(df_upcoming_game_encoded.columns)
for column in missing_columns:
    df_upcoming_game_encoded[column] = 0

# reordering columns to match the order during training
df_upcoming_game_encoded = df_upcoming_game_encoded[X_train_completed.columns]

# making predictions using the trained model for binary outcome
predicted_probabilities = model_completed.predict_proba(df_upcoming_game_encoded)

prediction_date = 'Monday 12th Feb 2024'

# ANSI escape codes for color
blue_color = '\033[94m'
reset_color = '\033[0m'
green_color = '\033[92m'
red_color = '\033[91m'

# data for PrettyTable
table_new = PrettyTable()
table_new.field_names = [f"{blue_color}NBA, {prediction_date}{reset_color}", f"{blue_color}Projected Winners, v1.1{reset_color}"]
table_new.align["Projected Winners"] = "l"
table_new.horizontal_char = '-'  # Use a horizontal line as a separator

# determining the average win percentage threshold for color coding
average_win_percentage_threshold = df_existing['Win'].value_counts(normalize=True).mean() * 100

for i, team in enumerate(df_upcoming_game.itertuples(), start=1):  # Starting from index 1 to skip the header row
    try:
        team1_win_percentage = df_existing[df_existing['Home team'] == team[1]]['Win'].value_counts(normalize=True).get(team[1], 0) * 100
        team2_win_percentage = df_existing[df_existing['Home team'] == team[2]]['Win'].value_counts(normalize=True).get(team[2], 0) * 100
    except Exception as e:
        print(f"Error calculating win percentage for teams {team[1]} and {team[2]}:", str(e))
        continue

    # Determine the team with the higher win percentage as the projected winner
    if team1_win_percentage > team2_win_percentage:
        predicted_winner = team[1]
        projected_win_percentage = team1_win_percentage
        predicted_winner_color = green_color if team1_win_percentage > average_win_percentage_threshold else red_color
    else:
        predicted_winner = team[2]
        projected_win_percentage = team2_win_percentage
        predicted_winner_color = green_color if team2_win_percentage > average_win_percentage_threshold else red_color

    team_info = f"{green_color}{team[1]} ({df_existing['Win'].eq(team[1]).sum()}-{df_existing['Loss'].eq(team[1]).sum()}){reset_color} vs {green_color}{team[2]} ({df_existing['Win'].eq(team[2]).sum()}-{df_existing['Loss'].eq(team[2]).sum()}){reset_color}"

    table_new.add_row([team_info, f"{predicted_winner_color}{predicted_winner} ({projected_win_percentage:.2f}%) {reset_color}"])

print(table_new)


+----------------------------------------------------------------+--------------------------------+
|                   NBA, Monday 12th Feb 2024                    |    Projected Winners, v1.1     |
+----------------------------------------------------------------+--------------------------------+
|      Indiana Pacers (30-25) vs Charlotte Hornets (11-41)       |    Indiana Pacers (60.71%)     |
|   Philadelphia 76ers (31-21) vs Cleveland Cavaliers (35-16)    | Cleveland Cavaliers (69.23%)   |
|         Chicago Bulls (25-28) vs Atlanta Hawks (24-29)         |    Chicago Bulls (55.56%)      |
|      San Antonio Spurs (10-43) vs Toronto Raptors (19-34)      |   Toronto Raptors (44.00%)     |
|       New York Knicks (33-20) vs Houston Rockets (23-29)       |   New York Knicks (70.37%)     |
|   New Orleans Pelicans (31-22) vs Memphis Grizzlies (18-35)    | New Orleans Pelicans (60.00%)  |
|       Denver Nuggets (36-17) vs Milwaukee Bucks (34-19)        |    Denver Nuggets (84.00%)     |


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from colorama import Fore, Style
from prettytable import PrettyTable

# Load the existing NBA data
df_existing = pd.read_excel('NBA2324.xlsx')

# Checking if the 'Win' column exists before proceeding
winning_team = 'Win'
if winning_team in df_existing.columns:
    # Create a binary target variable indicating whether the home team wins
    df_existing['HomeWin'] = (df_existing['Home team'] == df_existing[winning_team]).astype(int)

    # Specify the categorical columns for one-hot encoding
    categorical_columns = ['Away team', 'Home team', 'Overtime', 'Arena']

    # Check if categorical columns exist in the DataFrame
    missing_columns = [col for col in categorical_columns if col not in df_existing.columns]

    if not missing_columns:
        # Extract the necessary columns for the model
        X_completed = df_existing[categorical_columns]
        y_completed = df_existing['HomeWin']

        # One-hot encode categorical columns
        X_completed_encoded = pd.get_dummies(X_completed, drop_first=True)

        # Split the completed data into training and testing sets
        X_train_completed, _, y_train_completed, _ = train_test_split(
            X_completed_encoded, y_completed, test_size=0.2, random_state=42
        )

        # Choose a model for completed games (Logistic Regression in this example)
        model_completed = LogisticRegression()

        # Train the model on the completed data
        model_completed.fit(X_train_completed, y_train_completed)
    else:
        print(f"Error: Columns {missing_columns} not found in DataFrame.")
else:
    print(f"Error: Column '{winning_team}' not found in DataFrame.")

# Upcoming game data
upcoming_game_data = {
    'Away team': ['Oklahoma City Thunder', 'Detroit Pistons', 'San Antonio Spurs', 'Memphis Grizzlies', 'Chicago Bulls', 'Philadelphia 76ers', 'Houston Rockets', 'Indiana Pacers', 'Cleveland Cavaliers', 'Phoenix Suns', 'New Orleans Pelicans'],
    'Home team': ['Dallas Mavericks', 'Los Angeles Clippers', 'Brooklyn Nets', 'Charlotte Hornets', 'Orlando Magic', 'Washington Wizards', 'Atlanta Hawks', 'New York Knicks', 'Toronto Raptors', 'Golden State Warriors', 'Portland Trail Blazers'],
    'Overtime': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # Placeholder for upcoming games
    'Arena': ['American Airlines Center', 'Crypto.com Arena', 'Barclays Center', 'Spectrum Center', 'Amway Center', 'Capital One Arena', 'State Farm Arena', 'Madison Square Garden (IV)', 'Scotiabank Arena', 'Chase Center', 'Moda Center'],
}

# Create a DataFrame for the upcoming game
df_upcoming_game = pd.DataFrame(upcoming_game_data)

# Ensure the columns in df_upcoming_game_encoded match the columns used during training
df_upcoming_game_encoded = pd.get_dummies(df_upcoming_game)
missing_columns = set(X_train_completed.columns) - set(df_upcoming_game_encoded.columns)
for column in missing_columns:
    df_upcoming_game_encoded[column] = 0

# Reorder columns to match the order during training
df_upcoming_game_encoded = df_upcoming_game_encoded[X_train_completed.columns]

# Make predictions using the trained model for binary outcome
predicted_probabilities = model_completed.predict_proba(df_upcoming_game_encoded)

prediction_date = 'Saturday 10th Feb 2024'

# ANSI escape codes for color
blue_color = '\033[94m'
reset_color = '\033[0m'
green_color = '\033[92m'
red_color = '\033[91m'

# Data for PrettyTable
table_new = PrettyTable()
table_new.field_names = [f"{blue_color}NBA, {prediction_date}{reset_color}", f"{blue_color}Projected Winners, v1.1{reset_color}"]
table_new.align["Projected Winners"] = "l"
table_new.horizontal_char = '-'  # Use a horizontal line as a separator

# Determine the average win percentage threshold for color coding
average_win_percentage_threshold = df_existing['Win'].value_counts(normalize=True).mean() * 100

for i, team in enumerate(df_upcoming_game.itertuples(), start=1):  # Starting from index 1 to skip the header row
    try:
        team1_win_percentage = df_existing[df_existing['Home team'] == team[1]]['Win'].value_counts(normalize=True).get(team[1], 0) * 100
        team2_win_percentage = df_existing[df_existing['Home team'] == team[2]]['Win'].value_counts(normalize=True).get(team[2], 0) * 100
    except Exception as e:
        print(f"Error calculating win percentage for teams {team[1]} and {team[2]}:", str(e))
        continue

    # Determine the team with the higher win percentage as the projected winner
    if team1_win_percentage > team2_win_percentage:
        projected_winner = team[1]
        projected_win_percentage = team1_win_percentage
        projected_winner_color = green_color if team1_win_percentage > average_win_percentage_threshold else red_color
    else:
        projected_winner = team[2]
        projected_win_percentage = team2_win_percentage
        projected_winner_color = green_color if team2_win_percentage > average_win_percentage_threshold else red_color

    # Check if the projected winner's win percentage is greater than 50%
    if projected_win_percentage < 50:
        # Swap the projected winner and color
        projected_winner, projected_winner_color = team[1], green_color

    team_info = f"{green_color}{team[1]} ({df_existing['Win'].eq(team[1]).sum()}-{df_existing['Loss'].eq(team[1]).sum()}){reset_color} vs {green_color}{team[2]} ({df_existing['Win'].eq(team[2]).sum()}-{df_existing['Loss'].eq(team[2]).sum()}){reset_color}"

    table_new.add_row([team_info, f"{projected_winner_color}{projected_winner} ({projected_win_percentage:.2f}%) {reset_color}"])

print(table_new)

+----------------------------------------------------------------+---------------------------------+
|                  NBA, Saturday 10th Feb 2024                   |     Projected Winners, v1.1     |
+----------------------------------------------------------------+---------------------------------+
|   Oklahoma City Thunder (36-17) vs Dallas Mavericks (30-23)    | Oklahoma City Thunder (77.78%)  |
|     Detroit Pistons (8-44) vs Los Angeles Clippers (35-16)     |  Los Angeles Clippers (80.00%)  |
|       San Antonio Spurs (10-43) vs Brooklyn Nets (21-31)       |   San Antonio Spurs (44.83%)    |
|     Memphis Grizzlies (18-35) vs Charlotte Hornets (11-41)     |   Memphis Grizzlies (22.22%)    |
|         Chicago Bulls (25-28) vs Orlando Magic (29-24)         |     Orlando Magic (70.83%)      |
|    Philadelphia 76ers (31-21) vs Washington Wizards (9-43)     |   Philadelphia 76ers (62.96%)   |
|        Houston Rockets (23-29) vs Atlanta Hawks (24-29)        |    Houston Rockets (66.6

In [4]:
#tests

In [5]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from colorama import Fore, Style
# from prettytable import PrettyTable

# # Load the existing NBA data
# df_existing = pd.read_excel('NBA2324.xlsx')

# # Checking if the 'Win' column exists before proceeding
# winning_team = 'Win'
# if winning_team in df_existing.columns:
#     # Create a binary target variable indicating whether the home team wins
#     df_existing['HomeWin'] = (df_existing['Home team'] == df_existing[winning_team]).astype(int)

#     # Specify the categorical columns for one-hot encoding
#     categorical_columns = ['Away team', 'Home team', 'Overtime', 'Arena']

#     # Check if categorical columns exist in the DataFrame
#     missing_columns = [col for col in categorical_columns if col not in df_existing.columns]

#     if not missing_columns:
#         # Extract the necessary columns for the model
#         X_completed = df_existing[categorical_columns]
#         y_completed = df_existing['HomeWin']

#         # One-hot encode categorical columns
#         X_completed_encoded = pd.get_dummies(X_completed, drop_first=True)

#         # Split the completed data into training and testing sets
#         X_train_completed, _, y_train_completed, _ = train_test_split(
#             X_completed_encoded, y_completed, test_size=0.2, random_state=42
#         )

#         # Choose a model for completed games (Logistic Regression in this example)
#         model_completed = LogisticRegression()

#         # Train the model on the completed data
#         model_completed.fit(X_train_completed, y_train_completed)
#     else:
#         print(f"Error: Columns {missing_columns} not found in DataFrame.")
# else:
#     print(f"Error: Column '{winning_team}' not found in DataFrame.")

# # Upcoming game data
# upcoming_game_data = {
#     'Away team': ['Oklahoma City Thunder', 'Detroit Pistons', 'San Antonio Spurs', 'Memphis Grizzlies', 'Chicago Bulls', 'Philadelphia 76ers', 'Houston Rockets', 'Indiana Pacers', 'Cleveland Cavaliers', 'Phoenix Suns', 'New Orleans Pelicans'],
#     'Home team': ['Dallas Mavericks', 'Los Angeles Clippers', 'Brooklyn Nets', 'Charlotte Hornets', 'Orlando Magic', 'Washington Wizards', 'Atlanta Hawks', 'New York Knicks', 'Toronto Raptors', 'Golden State Warriors', 'Portland Trail Blazers'],
#     'Overtime': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # Placeholder for upcoming games
#     'Arena': ['American Airlines Center', 'Crypto.com Arena', 'Barclays Center', 'Spectrum Center', 'Amway Center', 'Capital One Arena', 'State Farm Arena', 'Madison Square Garden (IV)', 'Scotiabank Arena', 'Chase Center', 'Moda Center'],
# }

# # Create a DataFrame for the upcoming game
# df_upcoming_game = pd.DataFrame(upcoming_game_data)

# # Ensure the columns in df_upcoming_game_encoded match the columns used during training
# df_upcoming_game_encoded = pd.get_dummies(df_upcoming_game)
# missing_columns = set(X_train_completed.columns) - set(df_upcoming_game_encoded.columns)
# for column in missing_columns:
#     df_upcoming_game_encoded[column] = 0

# # Reorder columns to match the order during training
# df_upcoming_game_encoded = df_upcoming_game_encoded[X_train_completed.columns]

# # Make predictions using the trained model for binary outcome
# predicted_probabilities = model_completed.predict_proba(df_upcoming_game_encoded)

# prediction_date = 'Saturday 10th Feb 2024'

# # ANSI escape codes for color
# blue_color = '\033[94m'
# reset_color = '\033[0m'
# green_color = '\033[92m'
# red_color = '\033[91m'

# # Data for PrettyTable
# table_new = PrettyTable()
# table_new.field_names = [f"{blue_color}NBA, {prediction_date}{reset_color}", f"{blue_color}Projected Winners, v1.2{reset_color}"]
# table_new.align["Projected Winners"] = "l"
# table_new.horizontal_char = '-'  # Use a horizontal line as a separator

# # Determine the average win percentage threshold for color coding
# average_win_percentage_threshold = df_existing['Win'].value_counts(normalize=True).mean() * 100

# for i, team in enumerate(df_upcoming_game.itertuples(), start=1):  # Starting from index 1 to skip the header row
#     try:
#         team1_win_probability = predicted_probabilities[i - 1][1] * 100
#         team2_win_probability = 100 - team1_win_probability
#     except Exception as e:
#         print(f"Error calculating win probability for teams {team[1]} and {team[2]}:", str(e))
#         continue

#     # Determine the projected winner and its win probability
#     if team1_win_probability > 50:
#         projected_winner = team[1]
#         projected_win_probability = team1_win_probability
#     else:
#         projected_winner = team[2]
#         projected_win_probability = team2_win_probability

#     team_info = f"{green_color}{team[1]}{reset_color} vs {green_color}{team[2]}{reset_color}"

#     table_new.add_row([team_info, f"{green_color}{projected_winner} ({projected_win_probability:.2f}%) {reset_color}"])

# print(table_new)
